# NIMA: Neural Image Assessment
replicate model for NIMA image assessment
see: 
* https://research.googleblog.com/2017/12/introducing-nima-neural-image-assessment.html
* https://arxiv.org/abs/1709.05424



## Table of Contents
<a href="#Install">Installation and setup</a><br>

## Overview
### Components
tensorflow models for VGG16, Inception-v2, MobileNet
* https://github.com/tensorflow/models/tree/master/research/slim

training datasets
* AVA: A largescale database for aesthetic visual analysis
  * http://refbase.cvc.uab.es/files/MMP2012a.pdf
  * https://github.com/mtobeiyf/ava_downloader
  * https://mega.nz/#F!hIEhQTLY key `!RkOnZv8Fz7EbYreHsiEzvA` (32GB)
  * https://mega.nz/#!MUcXyBSB key `!0Q0Nq8_zBuSGiKmEHuKXKoAg8SDsB-21GwlJ22AJegU`
  
* TID2013: http://www.ponomarenko.info/tid2013.htm
  * http://www.ponomarenko.info/tid2013/tid2013.rar (1GB)

### Pipeline
* input images are rescaled to 256 × 256, and then a crop of size 224 × 224 crop is randomly extracted.
* random data augmentation in our training process is horizontal flipping of the image crops.

### Score
* mean quality score = `sum_N( s_i*p_i)`


### Loss function
* EMD (Earth Movers Distance) penalize mis-classifications according to class distances.
  * https://gist.github.com/mjdietzx/a8121604385ce6da251d20d018f9a6d6
  * https://www.tensorflow.org/api_docs/python/tf/distributions/Distribution
* CMD, cumulative distr function, N_ava=10, N_tid=9
    ```
    EMD(p,phat) = (1/N.*sum_k( abs(CDF_p(k)-CDF_phat(k)).^2 )).^0.5
    ```

### Training
* 80/20 train/test split on AVA and TID datasets

hyperparameters
  * `momentum=0.9, lambda=3e-7` 
  * `dropout=0.75` applied to last layer of baseline network

FC layer, n=10, followed by softmax activations
  * `lambda_fc=3e-6`

lambda `decay=0.95` after every 10 epochs

**???: how many epochs**


  

<a id='Install'></a>
## Installation and Setup

### Download datasets
* AVA dataset is 32GB, 256K images
* TID dataset is about 1GB, about 3K images


In [1]:
# set key paths
HOME = %pwd
DATA = HOME + '/data'
CKPT = HOME + '/ckpt'
TMP = HOME + '/tmp'
SLIM = HOME + '/models/research/slim'
# print(HOME, DATA, CKPT, SLIM)
AVA=DATA + '/ava'
TID=DATA + '/tid'

%mkdir DATA
%mkdir AVA
%mkdir TID

mkdir: DATA: File exists


In [2]:
%cd $DATA
# download tid, 1GB 
!wget http://www.ponomarenko.info/tid2013/tid2013.rar

/dev.snappi/tensorflow/a-lamp/data
--2017-12-28 17:30:39--  http://www.ponomarenko.info/tid2013/tid2013.rar
Resolving www.ponomarenko.info... 95.213.182.162
Connecting to www.ponomarenko.info|95.213.182.162|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957680241 (913M) [application/x-rar-compressed]
Saving to: 'tid2013.rar'

tid2013.rar           8%[>                   ]  78.81M  1.01MB/s    eta 9m 15s ^C


In [3]:
# rar archiver for python/conda, https://anaconda.org/pypi/unrar
!pip install -i https://pypi.anaconda.org/pypi/simple unrar
#!conda install -c mlgill rarfile 

### Tensorflow Models with Pre-trained Weights

In [4]:
%cd $HOME
!git clone https://github.com/tensorflow/models/

/dev.snappi/tensorflow/a-lamp
fatal: destination path 'models' already exists and is not an empty directory.


In [5]:
# check tf-slim install
%cd $SLIM
!python -c "from nets import cifarnet; mynet = cifarnet.cifarnet"


/dev.snappi/tensorflow/a-lamp/models/research/slim


In [6]:
%cd $SLIM
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
import time

from datasets import dataset_utils

# Main slim library
from tensorflow.contrib import slim

/dev.snappi/tensorflow/a-lamp/models/research/slim


In [7]:
# download tensorflow checkpoints, i.e. pre-trained weights
!mkdir -p $CKPT
!mkdir -p $TMP
%cd $TMP

### download model checkpoint
# vgg16
!wget http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz
!tar -xvf vgg_16_2016_08_28.tar.gz
%mv vgg_16.ckpt $CKPT
%rm vgg_16_2016_08_28.tar.gz

# inception-v2
!wget http://download.tensorflow.org/models/inception_v2_2016_08_28.tar.gz
!tar -xvf inception_v2_2016_08_28.tar.gz
%mv inception_v2.ckpt $CKPT
%rm inception_v2_2016_08_28.tar.gz

# MobileNet
!wget http://download.tensorflow.org/models/mobilenet_v1_1.0_224_2017_06_14.tar.gz
!tar -xvf mobilenet_v1_1.0_224_2017_06_14.tar.gz
%mv mobilenet_v1_1.0_224.ckpt.* $CKPT
%rm mobilenet_v1_1.0_224_2017_06_14.tar.gz  

/dev.snappi/tensorflow/a-lamp/tmp
--2017-12-28 17:32:04--  http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz
Resolving download.tensorflow.org... 216.58.211.112, 2a00:1450:400e:809::2010
Connecting to download.tensorflow.org|216.58.211.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 513324920 (490M) [application/x-tar]
Saving to: 'vgg_16_2016_08_28.tar.gz'

vgg_16_2016_08_28.t 100%[===================>] 489.54M  3.12MB/s    in 2m 47s  

2017-12-28 17:34:53 (2.93 MB/s) - 'vgg_16_2016_08_28.tar.gz' saved [513324920/513324920]

x vgg_16.ckpt
--2017-12-28 17:35:17--  http://download.tensorflow.org/models/inception_v2_2016_08_28.tar.gz
Resolving download.tensorflow.org... 216.58.211.112, 2a00:1450:400e:809::2010
Connecting to download.tensorflow.org|216.58.211.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41577079 (40M) [application/x-tar]
Saving to: 'inception_v2_2016_08_28.tar.gz'

inception_v2_2016_0 100%[==========

In [8]:
%ls $DATA

AVA_dataset/     ava/             tid2013.rar
AVA_dataset.zip* tid/


In [9]:
%cd $DATA
# OSX: brew install unrar
# manually extract archive
# unrar e tid2013.rar ./tid
# %rm $DATA/tid2013.rar


/dev.snappi/tensorflow/a-lamp/data
